In [1]:
import pandas as pd
from alpha_vantage.timeseries import TimeSeries

In [70]:
class ScriptData:
    def __init__(self):
        self.data={}
    def fetch_intraday_data(self,symbol):
        self.symbol = symbol
        ts = TimeSeries(key='TZC4MVZ8NEY5J1WJ')
        self.data, meta_data = ts.get_intraday(self.symbol)
        # print(self.data)
    def convert_intraday_data(self,symbol):

        df = pd.DataFrame(columns = ["Timestamp" , "Open" , "High" , "Low" , "Close" , "Volumn"])
        for i in self.data:
            df = df.append({"Timestamp" : pd.Timestamp(i), "Open" : self.data[i]["1. open"] , "High" : self.data[i]["2. high"] , "Low" : self.data[i]["3. low"] , "Close" : self.data[i]["4. close"] , "Volumn" : self.data[i]["5. volume"]},ignore_index=True)
        
        df["Open"] = df["Open"].astype(float)
        df["High"] = df["High"].astype(float)
        df["Low"] = df["Low"].astype(float)
        df["Close"] = df["Close"].astype(float)
        df["Volumn"] = df["Volumn"].astype(int)
        return(df)
        

In [ ]:
script_data = ScriptData()
script_data.fetch_intraday_data("GOOGL")
script_df = script_data.convert_intraday_data("GOOGL")
script_df

In [72]:
def indicator1(inp_df , timeperiod):
    out_df = pd.DataFrame(columns=["Timestamp" , "Indicator"])
    out_df["Timestamp"] = inp_df["Timestamp"]
    out_df["Indicator"] = inp_df["Close"].rolling(timeperiod).mean()
    return(out_df)

In [ ]:
indicator1(script_df,timeperiod=5)

In [89]:
class Strategy(ScriptData):
    def get_script_data(self,symbol):
        self.symbol = symbol 
        super().fetch_intraday_data(self.symbol)
        super().convert_intraday_data(self.symbol)
    def get_signal(self):
        self.fetch_intraday_data(self.symbol)
        self.intraday_data_df = self.convert_intraday_data(self.symbol)
        timestamp = list(self.intraday_data_df["Timestamp"])
        close_data = list(self.intraday_data_df["Close"])
        
        indicator_df =list(indicator1(self.intraday_data_df,timeperiod=5)["Indicator"])
        
        signal_list = []
        for i in range(len(indicator_df)):
            if(round(close_data[i],0) == round(indicator_df[i],0) and round(close_data[i],0) ) < round(indicator_df[i],0):
                signal_list.append("BUY")
            elif(round(close_data[i],0) == round(indicator_df[i],0) and round(close_data[i],0) ) < round(indicator_df[i],0):
                signal_list.append("SELL")
            else:
                signal_list.append("NO SIGNAL")
        
        signals = pd.DataFrame(list(zip(timestamp,signal_list)),columns=["Timestamp","Signal"])
        
        signals = signals[signals["Signal"]!="NO SIGNAL"]
        return(signals)


In [ ]:
a = Strategy()
a.get_script_data("GOOGL")
a.get_signal()